In [1]:
from sklearn.linear_model import ElasticNet, ElasticNetCV, Lasso
from TimewiseKfold import splitting_data, split_train_test, FOLD_NUM, MAXIMUM_YEAR
from Metrics import metrics_calculation, metrics_table
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
folds, PROVINCES_NUM = splitting_data(is_NDVI=True, region=2)

In [3]:
table = metrics_table()
for (i, fold) in enumerate(folds):
    X_train, X_test, y_train, y_test = split_train_test(fold=fold)
    
    linear_model = LinearRegression(n_jobs=-1).fit(X=X_train, y=y_train)
    
    row = [MAXIMUM_YEAR - FOLD_NUM + 1 + i]
    
    row += metrics_calculation(y_actual=y_test, y_pred=linear_model.predict(X_test), y_train=y_train)
    
    table.loc[len(table)] = row
    
    del linear_model
table

,Year,RMSE,R2_Score,MSE,MAE,RAE,RSE
0,2018.0,1.352236,0.380868,1.828542,1.140736,0.795713,0.673609
1,2019.0,1.681056,-0.777001,2.825951,1.435342,1.389500,1.786395
2,2020.0,5.884527,0.035338,34.627664,3.826344,1.030156,1.352279


In [4]:
selected_ndvi = ['NDVI_V', 'NDVI_X']

In [5]:
table = metrics_table()
for (i, fold) in enumerate(folds):
    X_train, X_test, y_train, y_test = split_train_test(fold=fold)
    
    X_train = X_train[selected_ndvi]
    X_test = X_test[selected_ndvi]
    
    linear_model = LinearRegression(n_jobs=-1).fit(X=X_train, y=y_train)
    
    row = [MAXIMUM_YEAR - FOLD_NUM + 1 + i]
    
    row += metrics_calculation(y_actual=y_test, y_pred=linear_model.predict(X_test), y_train=y_train)
    
    table.loc[len(table)] = row
    
    del linear_model
table

,Year,RMSE,R2_Score,MSE,MAE,RAE,RSE
0,2018.0,1.373692,0.361065,1.887029,1.100961,0.767969,0.695154
1,2019.0,1.602697,-0.615198,2.568637,1.362128,1.318625,1.623736
2,2020.0,6.008797,-0.005835,36.105636,3.680607,0.990920,1.409997


In [6]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import ElasticNet

tscv = TimeSeriesSplit(n_splits=2, test_size=PROVINCES_NUM)

OwO = []

table = metrics_table()

for (i, fold) in enumerate(folds):
    X_train, X_test, y_train, y_test = split_train_test(fold=fold)
    
    elastic_cv = ElasticNet()
    elastic_cv.fit(X_train, y_train)
    
    OwO.append(elastic_cv.coef_)
    print(elastic_cv.coef_)
    
    row = [MAXIMUM_YEAR - FOLD_NUM + 1 + i]
    
    row += metrics_calculation(y_actual=y_test, y_pred=elastic_cv.predict(X_test), y_train=y_train)
    
    table.loc[len(table)] = row
    
    del elastic_cv

X_train, X_test, y_train, y_test = split_train_test(fold=folds[0])

selected_features = []
for (i, factor) in enumerate(X_train.columns):
    if (OwO[1][i] != 0):
        selected_features.append(factor)
    
print(selected_features)
table

[ 0.  0. -0. -0.  0. -0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0.  0.]
[-0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0.  0.]
[]


,Year,RMSE,R2_Score,MSE,MAE,RAE,RSE
0,2018.0,1.718545,0.0,2.953397,1.348036,0.940315,1.087989
1,2019.0,1.261068,0.0,1.590292,1.074558,1.040239,1.005286
2,2020.0,5.991341,0.0,35.896168,3.529559,0.950254,1.401817


In [7]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=2, test_size=PROVINCES_NUM)

OwO = []

table = metrics_table()

for (i, fold) in enumerate(folds):
    X_train, X_test, y_train, y_test = split_train_test(fold=fold)
    
    elastic_cv = ElasticNetCV(l1_ratio=[0.1, 0.2, 0.3, 0.5, 0.75, 0.9, 0.99, 1], cv=tscv, max_iter=20000, n_jobs=-1)
    elastic_cv.fit(X_train, y_train)
    
    OwO.append(elastic_cv.coef_)
    print(elastic_cv.coef_)
    
    row = [MAXIMUM_YEAR - FOLD_NUM + 1 + i]
    
    row += metrics_calculation(y_actual=y_test, y_pred=elastic_cv.predict(X_test), y_train=y_train)
    
    table.loc[len(table)] = row
    
    del elastic_cv

X_train, X_test, y_train, y_test = split_train_test(fold=folds[0])

selected_features = []
for (i, factor) in enumerate(X_train.columns):
    if (OwO[2][i] != 0):
        selected_features.append(factor)
    
print(selected_features)
table

c:\Application\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[ 0.  0. -0. -0.  0. -0.  0.  0.  0.  0.  0.  0.]


c:\Application\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[ 1.48277854 -1.04080156  0.58889243 -5.80179107 10.56702701 -8.02484453
  2.34199243  2.09327074 -5.87143688 10.86638587 -4.25399751  0.02739267]


c:\Application\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1623: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[ 0.          0.          0.          0.          0.50207527 -0.
 -0.         -0.          0.          2.11535699 -0.         -0.        ]
['NDVI_V', 'NDVI_X']


,Year,RMSE,R2_Score,MSE,MAE,RAE,RSE
0,2018.0,1.718545,0.000000,2.953397,1.348036,0.940315,1.087989
1,2019.0,1.673648,-0.761373,2.801099,1.430427,1.384743,1.770685
2,2020.0,6.069871,-0.026386,36.843332,3.641960,0.980515,1.438805
